## 1) Lets get our dataset from google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/Data/CV_proj"

In [ ]:
!unzip -q /content/drive/MyDrive/Data/CV_proj/data.zip
#/content/drive/MyDrive/Data/Trays

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import PIL
from sklearn.model_selection import train_test_split
import torch
import torchvision
# from torch_snippets import Report
import os
import time
import xml.etree.ElementTree as ET
# Connect to the GPU if one exists.
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
import cv2
from google.colab.patches import cv2_imshow
import torch
import torchvision
import numpy as np
import pandas as pd
import os
import cv2
import time
import matplotlib.pyplot as plt

from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset, DataLoader
from torch import nn

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

from torchvision.ops.boxes import nms

In [ ]:
# !pip install opendatasets
# !pip install pandas

## 2) Now we need to convert our data to dataframe

In [ ]:
def my_xml_to_dict(xml_path):
    # Decode the .xml file
    tree = ET.parse(xml_path)
    root = tree.getroot()
    # Return the image size, object label and bounding box
    # coordinates together with the filename as a dict.
    names = []
    for tag in root.findall('object/name'):
        names.append(tag.text)

    x_mins = []
    for tag in root.findall('object/bndbox/xmin'):
        x_mins.append(tag.text)

    x_maxs = []
    for tag in root.findall('object/bndbox/xmax'):
        x_maxs.append(tag.text)

    y_mins = []
    for tag in root.findall('object/bndbox/ymin'):
        y_mins.append(tag.text)

    y_maxs = []
    for tag in root.findall('object/bndbox/ymax'):
        y_maxs.append(tag.text)

    return {"filename": xml_path,
            "image_width": int(root.find("./size/width").text),
            "image_height": int(root.find("./size/height").text),
            "image_channels": int(root.find("./size/depth").text),
            "label": names,
            "x1": x_mins,
            "y1": y_mins,
            "x2": x_maxs,
            "y2": y_maxs}

In [ ]:
# Convert human readable str label to int.
label_dict = {"mandarin": 1, "apple" : 2, "banana" : 3, "pepper" : 4, "cucumber" : 5, "carrot" : 6}
# Convert label int to human readable str.
reverse_label_dict = {1: "mandarin", 2: "apple", 3: "banana", 4: "pepper", 5 : "cucumber", 6: "carrot"}

In [ ]:
xml_path  =   "/content/train_xml/"+ "photo_10_2023-12-20_14-14-22" + ".xml"
my_xml_to_dict(xml_path)

In [ ]:
def my_show_with_box(im_name, dir_path, xml_path):
    xml_path  =   xml_path+ im_name + ".xml"
    path_to_img = dir_path+ im_name + ".jpg"
    img = cv2.imread(path_to_img)
    for i in range(len(my_xml_to_dict(xml_path)['x1'])):
        x1 = int(my_xml_to_dict(xml_path)['x1'][i])
        x2 = int(my_xml_to_dict(xml_path)['x2'][i])
        y1 = int(my_xml_to_dict(xml_path)['y1'][i])
        y2 = int(my_xml_to_dict(xml_path)['y2'][i])
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
        print(my_xml_to_dict(xml_path)['label'][i]," :", x1, x2, y1, y2)
    #cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
    cv2_imshow(img)
my_show_with_box("photo_10_2023-12-20_14-14-22",  "/content/train/","/content/train_xml/")

In [ ]:
def labelling(path_data, path_xml):
    label_list = []
    for i in range(len(sorted(os.listdir(path_data)))):
        xml_path  =   path_xml+ sorted(os.listdir(path_xml))[i]
        labels_name_list=my_xml_to_dict(xml_path)['label']
        labels_fig_list = list()
        for i in range(len(labels_name_list)):
            labels_fig_list.append(int(label_dict[labels_name_list[i]]))
        label_list.append(labels_fig_list)
    all_boxes = list()
    for i in range(len(sorted(os.listdir(path_data)))):
        xml_path  =   path_xml+ sorted(os.listdir(path_xml))[i]
        x1 = my_xml_to_dict(xml_path)['x1']
        x2 = my_xml_to_dict(xml_path)['x2']
        y1 = my_xml_to_dict(xml_path)['y1']
        y2 = my_xml_to_dict(xml_path)['y2']
        boxes = list()
        for i in range(len(x1)):
            boxes.append([int(x1[i]), int(y1[i]), int(x2[i]), int(y2[i])])
        all_boxes.append(boxes)
    d = {'filename': sorted(os.listdir(path_data)), 'class': label_list, 'bbox': all_boxes}
    return pd.DataFrame(data=d)

In [ ]:
#!rm  /content/train/aug*

In [ ]:
df_train = labelling("/content/train/", "/content/train_xml/")

In [ ]:
df_train

## 3) Augmentations

In [ ]:
import albumentations as A
import cv2
import copy
count=0

transform_rot90 = A.Compose([A.augmentations.geometric.transforms.Affine(rotate=90)], bbox_params=A.BboxParams(format='pascal_voc'))
transform_rot180 = A.Compose([A.augmentations.geometric.transforms.Affine(rotate=180)], bbox_params=A.BboxParams(format='pascal_voc'))
transform_random_bright = A.Compose([A.RandomBrightnessContrast(p=1)], bbox_params=A.BboxParams(format='pascal_voc'))
transform_horizontal_flip = A.Compose([A.HorizontalFlip(p=1)], bbox_params=A.BboxParams(format='pascal_voc'))

for num_sample in range(len(df_train)):
#for num_sample in range(2):
    trans_im_arr= list()
    trans_box_arr= list()

    #im_name = "IMG_20230918_130424"

    a = list(df_train['filename'][[num_sample]])
    im_name = a[0]

    path_to_img = "/content/train/"+ im_name
    image = cv2.imread(path_to_img)

    a = df_train['bbox'][[num_sample]]
    a = list(a)
    b = copy.deepcopy(a)
    #print(a[0])
    #b[0].append('sdf')
    box = b[0]
    #print(len(box))
    for i in range(len(box)):
        box[i].append("dog")
    #print(box)
    #box=[[149, 888, 836, 1563, 'dog']]
    transformed1 = transform_rot90(image=image, bboxes=box)
    transformed_image1 = transformed1['image']
    transformed_bboxes1 = transformed1['bboxes']
    trans_im_arr.append(transformed_image1)
    trans_box_arr.append(transformed_bboxes1)

    transformed2 = transform_rot180(image=image, bboxes=box)
    transformed_image2 = transformed2['image']
    transformed_bboxes2 = transformed2['bboxes']
    trans_im_arr.append(transformed_image2)
    trans_box_arr.append(transformed_bboxes2)

    transformed3 = transform_horizontal_flip(image=image, bboxes=box)
    transformed_image3 = transformed3['image']
    transformed_bboxes3 = transformed3['bboxes']
    trans_im_arr.append(transformed_image3)
    trans_box_arr.append(transformed_bboxes3)

    transformed4 = transform_horizontal_flip(image=transformed_image1, bboxes=transformed_bboxes1)
    transformed_image4 = transformed4['image']
    transformed_bboxes4 = transformed4['bboxes']
    trans_im_arr.append(transformed_image4)
    trans_box_arr.append(transformed_bboxes4)

    transformed5 = transform_horizontal_flip(image=transformed_image2, bboxes=transformed_bboxes2)
    transformed_image5 = transformed5['image']
    transformed_bboxes5 = transformed5['bboxes']
    trans_im_arr.append(transformed_image5)
    trans_box_arr.append(transformed_bboxes5)


    transformed6 = transform_random_bright(image=image, bboxes=box)
    transformed_image6 = transformed6['image']
    transformed_bboxes6 = transformed6['bboxes']
    trans_im_arr.append(transformed_image6)
    trans_box_arr.append(transformed_bboxes6)

    for i in range(5):
        transformed = transform_random_bright(image=trans_im_arr[i], bboxes=trans_box_arr[i])
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']
        trans_im_arr.append(transformed_image)
        trans_box_arr.append(transformed_bboxes)

    for i in range(len(trans_im_arr)): # цикл по 11 ауг картинкам
        #print(i)
        count+=1
        new_box = list()
        for j in range(len(trans_box_arr[i])): #цикл по рамкам
            x1 = int(trans_box_arr[i][j][0])
            x2 = int(trans_box_arr[i][j][2])
            y1 = int(trans_box_arr[i][j][1])
            y2 = int(trans_box_arr[i][j][3])
            #cv2.rectangle(trans_im_arr[i], (x1, y1), (x2, y2), (255, 0, 0), 3)
            new_box.append(list(trans_box_arr[i][j][:4]))
        #cv2_imshow(trans_im_arr[i])
        cv2.imwrite("/content/train/aug"+str(count)+".jpg", trans_im_arr[i]) # Save aug image as JPG file
        f_name = "aug"+str(count)+".jpg"
        new_row = {'filename':f_name, 'class':list(df_train['class'][[num_sample]])[0], 'bbox':new_box}
        df_train = df_train.append(new_row, ignore_index=True)

In [ ]:
df_train

Lets show our augmented data

In [ ]:
def my_show_with_box_for_aug_df(num):
    im_name = list(df_train['filename'][[num]])[0]
    path_to_img = "/content/train/"+ im_name
    print(path_to_img)
    img = cv2.imread(path_to_img)
    for i in range(len(list(df_train['bbox'][[num]])[0])):
        x1 = int(list(df_train['bbox'][[num]])[0][i][0])
        x2 = int(list(df_train['bbox'][[num]])[0][i][2])
        y1 = int(list(df_train['bbox'][[num]])[0][i][1])
        y2 = int(list(df_train['bbox'][[num]])[0][i][3])
        print(x1,x2,y1,y2)
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
        #print(my_xml_to_dict(xml_path)['label'][i]," :", x1, x2, y1, y2)
    # cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
    cv2_imshow(img)
my_show_with_box_for_aug_df(1006)
#my_show_with_box("aug3090",  "/content/train/aug","/content/train_xml/")


In [ ]:
df_val = labelling("/content/test/", "/content/test_xml/")

In [ ]:
df_val

In [ ]:
def draw_img_with_box(df_index, dir_path, scale_percent=20):
    path_to_img = os.path.join(dir_path, df_train.loc[df_index, 'filename'])
    print(path_to_img)
    img = cv2.imread(path_to_img)
    boxes =  df_train.loc[df_index, 'bbox']
    for box in boxes:
        cv2.rectangle(img,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            (255, 0, 0), 5)
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img = cv2.resize(img, dim)
    cv2_imshow(img)

draw_img_with_box(10, "/content/train" )

## 4) Lets create custom dataset class

In [ ]:
class MyDataset(Dataset):
    def __init__(self, dataframe, img_path):

        self.df = dataframe
        self.img_path = img_path

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        img_name = self.df.loc[index,'filename']
        boxes = torch.Tensor(self.df.loc[index, 'bbox']).to(torch.float)
        labels = torch.Tensor(self.df.loc[index, 'class']).to(torch.int64)
        area = (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 0])
        iscrowd = torch.zeros(labels.shape[0], dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels

        img = cv2.imread(os.path.join(self.img_path, img_name))/255.
        img = torch.from_numpy(img).permute(2, 0, 1).to(torch.float)
        return img, target

## 5) Model creation

In [ ]:
def create_model(num_classes, pretrained=False):
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=pretrained)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = create_model(num_classes=7, pretrained=False).to(device)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9, weight_decay=0.0005)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)

train_dataset = MyDataset(df_train, '/content/train')
val_dataset = MyDataset(df_val, '/content/test')

train_data_loader = DataLoader(
    train_dataset,
    batch_size=3,
    shuffle=True,
    collate_fn=collate_fn
)

val_data_loader = DataLoader(
    val_dataset,
    batch_size=3,
    shuffle=False,
    collate_fn=collate_fn
)

## 6) Lets train our Faster-RCNN

In [ ]:
def train(train_dataloader):
    model.train()
    running_loss = 0
    for i, data in enumerate(train_dataloader):
        optimizer.zero_grad()
        images, targets = data[0], data[1]
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
        if i % 50 == 0:
            print(f"\tИтерация #{i} loss: {loss}")
    train_loss = running_loss/len(train_dataloader.dataset)
    return train_loss

def val(val_dataloader):
    running_loss = 0
    for data in val_dataloader:
        optimizer.zero_grad()
        images, targets = data[0], data[1]
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.no_grad():
            loss_dict = model(images, targets)
        loss_dict = model(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        running_loss += loss.item()
    val_loss = running_loss/len(val_dataloader.dataset)
    return val_loss

In [ ]:
train_losses = []
val_losses = []
try:
    for epoch in range(7):
        start = time.time()
        train_loss = train(train_data_loader)
        val_loss = val(val_data_loader)
        scheduler.step()
        print(f"Эпоха #{epoch} train_loss: {train_loss}, val_loss: {val_loss}")
        end = time.time()
        print(f"Потрачено {round((end - start) / 60, 1)} минут на {epoch} эпоху")
        train_losses.append(train_loss)
        val_losses.append(val_loss)
except KeyboardInterrupt:
    print('Прервано пользователем')

## 7) Loss vizualization

In [ ]:
fig, ax  = plt.subplots()
ax.plot(train_losses, label='Train')
ax.plot(val_losses, label='Val')
ax.set(xlabel='Epoch', ylabel='Loss')
ax.legend()

## 8) Inference

In [ ]:
df_index = 3
model.eval()
img = cv2.imread(os.path.join('/content/test', df_val.loc[df_index, 'filename']))
img_ = img / 255.
img_ = torch.from_numpy(img_).permute(2, 0, 1).unsqueeze(0).to(torch.float).to(device)
predict = model(img_)

In [ ]:
predict

In [ ]:
def draw_predict(df_index, iou_threshold=0.1, threshold=0.8, scale_percent=25):
    model.eval()
    img = cv2.imread(os.path.join('/content/test', df_val.loc[df_index, 'filename']))
    print(df_val.loc[df_index, 'filename'])
    img_ = img / 255.
    img_ = torch.from_numpy(img_).permute(2, 0, 1).unsqueeze(0).to(torch.float).to(device)
    predict = model(img_)
    #print(predict)
    #int(predict[0]['labels'][0])
    #label_dict = {"fruit drink": 1, "cabbage salad" : 2, "soup" : 3}
    ind = nms(predict[0]['boxes'], predict[0]['scores'], iou_threshold).detach().cpu().numpy()
    for i, box in enumerate(predict[0]['boxes'][ind]):
        #print(i)
        color = (0,255,0)
        if predict[0]['scores'][i] > threshold:
            if int(predict[0]['labels'][i]) == 1: #fruit drink
                color = (176,31,186)
            if int(predict[0]['labels'][i]) == 2: #cabbcage salad
                color = (0,255,0)
            if int(predict[0]['labels'][i]) == 3: #mushrooms soup
                color = (47,77,26)

            cv2.rectangle(img,
                    (int(box[0]), int(box[1])),
                    (int(box[2]), int(box[3])),
                    color, 5)
            font                   = cv2.FONT_HERSHEY_SIMPLEX
            bottomLeftCornerOfText = (int(box[0]), int(box[1]))
            fontScale              = 5
            fontColor              = (255,255,255)
            thickness              = 10
            lineType               = 2
            cv2.putText(img, reverse_label_dict[int(predict[0]['labels'][i])], bottomLeftCornerOfText, font, fontScale, fontColor, thickness, lineType)


    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    dim = (width, height)
    img = cv2.resize(img, dim)
    cv2_imshow(img)
draw_predict(3, 0.1)
#bill_calculate(18, 0.1)

In [ ]:
torch.save(model.state_dict(), 'fasterrcnn_resnet50_fpn_5ka.pth')

In [ ]:
!cp "/content/fasterrcnn_resnet50_fpn_5ka.pth" /content/drive/MyDrive/Data/CV_proj

## 9) Accuraccy

In [ ]:
import xml.etree.ElementTree as ET

def read_xml(xml_path):
    # Decode the .xml file
    tree = ET.parse(xml_path)
    root = tree.getroot()
    # Return the image size, object label and bounding box
    # coordinates together with the filename as a dict.
    names = []

    for tag in root.findall('object/name'):
        names.append(tag.text)

    x_mins = []
    for tag in root.findall('object/bndbox/xmin'):
        x_mins.append(tag.text)

    x_maxs = []
    for tag in root.findall('object/bndbox/xmax'):
        x_maxs.append(tag.text)

    y_mins = []
    for tag in root.findall('object/bndbox/ymin'):
        y_mins.append(tag.text)

    y_maxs = []
    for tag in root.findall('object/bndbox/ymax'):
        y_maxs.append(tag.text)

    boxes = []
    for i in range(len(x_mins)):
        boxes.append([int(x_mins[i]), int(y_mins[i]), int(x_maxs[i]), int(y_maxs[i])])

    return {"filename": xml_path,
            "image_width": int(root.find("./size/width").text),
            "image_height": int(root.find("./size/height").text),
            "image_channels": int(root.find("./size/depth").text),
            "label": names,
            "box": boxes,
            "x1": x_mins,
            "y1": y_mins,
            "x2": x_maxs,
            "y2": y_maxs}


In [ ]:
import numpy as np

def get_iou(ground_truth, pred):
    # coordinates of the area of intersection.
    ix1 = np.maximum(ground_truth[0], pred[0])
    iy1 = np.maximum(ground_truth[1], pred[1])
    ix2 = np.minimum(ground_truth[2], pred[2])
    iy2 = np.minimum(ground_truth[3], pred[3])

    # Intersection height and width.
    i_height = np.maximum(iy2 - iy1 + 1, np.array(0.))
    i_width = np.maximum(ix2 - ix1 + 1, np.array(0.))

    area_of_intersection = i_height * i_width

    # Ground Truth dimensions.
    gt_height = ground_truth[3] - ground_truth[1] + 1
    gt_width = ground_truth[2] - ground_truth[0] + 1

    # Prediction dimensions.
    pd_height = pred[3] - pred[1] + 1
    pd_width = pred[2] - pred[0] + 1

    area_of_union = gt_height * gt_width + pd_height * pd_width - area_of_intersection

    iou = area_of_intersection / area_of_union

    return iou

import torch
from torch import tensor
from torchvision import ops

ops.box_iou(tensor([[72, 373, 708, 839]]), tensor([[80, 300, 710, 800]]))

In [ ]:
def calc_scores(img_xml, predict, threshold=0.8):
    results = []
    for i, score in enumerate(predict['scores']):
        if score > threshold:
            label_hat = predict['labels'][i]
            box_hat = predict['boxes'][i]

            scores = []
            for j, label in enumerate(img_xml['label']):
                if label_dict[label] == label_hat:
                    scores.append(get_iou(img_xml['box'][j], box_hat.detach().numpy()))

            iou = float(max(scores, default=0))
            results.append( (iou > 0.5) * 1 )

    return results

In [ ]:
#model loading
model = create_model(num_classes=7, pretrained=False).to(device)
state_dict = torch.load('/content/drive/MyDrive/Data/CV_proj/fasterrcnn_resnet50_fpn_5ka.pth', map_location=torch.device('cpu'))
model.load_state_dict(state_dict)


In [ ]:
label_dict = {"mandarin": 1, "apple" : 2, "banana" : 3, "pepper" : 4, "cucumber" : 5, "carrot" : 6}

img_name = "photo_2023-12-18_12-43-15.jpg"

predict = {'boxes': tensor([[649.7993, 160.8598, 977.5370, 490.6462],
          [636.0603, 521.2333, 913.5746, 847.4760],
          [644.7803, 161.2805, 972.5023, 482.9593],
          [632.1509, 519.2692, 894.2546, 847.7626]]),
  'labels': tensor([2, 2, 4, 4]),
  'scores': tensor([0.9737, 0.9708, 0.0596, 0.0542])
}

xml_path  = f"/content/test_xml/{img_name.split('.jpg')[0]}.xml"
img_xml = read_xml(xml_path)

calc_scores(img_xml, predict)

In [ ]:
c=0
summ = 0.
all = 0
for i in df_val['filename']:
    # if c == 12:
    #     break
    c+=1
    model.eval()
    img_name = i
    img = cv2.imread(os.path.join('/content/test', img_name))
    img_ = img / 255.
    img_ = torch.from_numpy(img_).permute(2, 0, 1).unsqueeze(0).to(torch.float).to(device)
    predict = model(img_)[0]
    #print(predict)
    xml_path  = f"/content/test_xml/{img_name.split('.jpg')[0]}.xml"
    img_xml = read_xml(xml_path)
    summ += np.sum(calc_scores(img_xml, predict))
    all += len(calc_scores(img_xml, predict))
    acc = summ/all
    print("№",c," -- ", "calculated score: ", calc_scores(img_xml, predict), "---","cumulated accuracy:", summ/all)
    if np.sum(calc_scores(img_xml, predict)) != len(calc_scores(img_xml, predict)):
        print("Not exact label: ",img_name)



Lets see pictures with incorrect labels:

In [ ]:
draw_predict(7, 0.1)

In [ ]:
draw_predict(16, 0.1)

In [ ]:
draw_predict(17, 0.1)

In [ ]:
draw_predict(21, 0.1)

In [ ]:
draw_predict(22, 0.1)

In [ ]:
draw_predict(7, 0.1)